# Ganymede | *RSA Encryption Cypher*

In [1]:
'''
RSA Encryption
'''
import numpy as np
import math as m
import random
import heapq

'''
Extended Euclid's algorithm for determining the greatest common divisor for large numbers
'''
def egcd(a, b):
    x, y, u, v = 0, 1, 1, 0
    while a != 0:
        q, r = b // a, b % a
        m, n = x - u * q, y - v * q
        b, a, x, y, u, v = a, r, u, v, m, n
    gcd = b
    return gcd, x, y

'''
Finds the modular inverse -
This is specifically used to find the decryption key: d = (phi*i + 1)/e
'''
def modinv(e, n):
    gcd, x, y = egcd(e, n)
    if gcd != 1:
        return None  # The modular inverse does not exist in this case
    else:
        return x % n

'''
Primality test: Tests values to determine if they are prime or not
Sieve of Eratosthenes: When non-prime, this will give all prime values less than the non-prime
'''
# No longer used for isPrime, but saved for possible future use.
# def isComposite(a, d, n, s):
#     if pow(a, d, n) == 1:
#         return False
#     for i in range(s):
#         if pow(a, 2**i * d, n) == n-1:
#             return False
#     return True

# For more information on primes see http://primes.utm.edu/
# The Prime Pages - maintained by Professor Chris Caldwell
def isPrime(n):
    if n <= 9999999999999999999999999999999999999999:
        if (n <= 1):
            return False
        if (n <= 3):
            return True
        if (n % 2 == 0 or n % 3 == 0):

            return False
        i = 3
        while (i*i <= n):
            if (n % i == 0 or n % (i + 2) == 0):
                return False
            i += 1
        return True
    else:
        print('WARNING: May not be prime.')
        return True

# Only works with numbers <= 200. Otherwise it hangs - removed and replaced with fixed samples of primes
# def sieveOfEratosthenes(n):
#     if n > 200:
#         n = 200
#     if 0 < n <= 200:
#         list = [True for i in range(n + 1)]
#         p, templist, primelist = 2, [], []
#         while pow(p, p) <= n:

#             # If list[p] is not changed, then it is a prime
#             if list[p] == True:

#                 # Update all multiples of p
#                 for i in range(p * 2, n + 1, p):
#                     list[i] = False
#                     if list[i] % p == 0:
#                         list[i] = False

#                 p += 1
#         list[0] = False
#         list[1] = False

#         # Print all prime numbers
#         for idx in range(n-1):
#             if list[idx] == True:
#                 templist += [idx]
#         for idx in templist:
#             if isPrime(idx) == True and idx > 30:
#                 primelist += [str(idx),' ']

#     return ''.join(primelist)

'''
Finds the modulus value
'''
def find_modulus(p, q):
    mod_n = (p-1)*(q-1)
    return mod_n

'''
Generate Public and Private Keys
'''
def keys(p, q, e):
    n, d = (p * q), modinv(e, find_modulus(p, q))

    return e, d, n

'''
Encryption
'''
def encrypt(keypack, plaintext):
    key, n = keypack

    cipher = [(ord(char)) for char in plaintext]
    # print('Step 1: ', cipher) # Converts each character into its ASCii value

    # Encrypts each block by converting ASCii values based on a^b mod m
    cipher = [str(pow(int(char), key, n)) for char in cipher]

    # The for loop will add leading zeros and buffers as necessary
    for idx in range(len(cipher)):
        cipher[idx] = str(cipher[idx])
        while len(str(cipher[idx])) < (len(str(n))):
            cipher[idx] = '0%s' % cipher[idx]
        if len(cipher) % 2 == 1:
            cipher.append('    ')

    # # print('Step 2:', cipher) # Combines blocks into largest size block possible but no larger than n; where n=p*q
    ciphertext = [i + j for i, j in zip(cipher[::2], cipher[1::2])]

    # print('Step 3:', ciphertext) # Combines pairs of n blocks
    ciphertext = ''.join(ciphertext)

    return ciphertext

'''
Decryption
'''
def decrypt(keypack, ciphertext):
    key, n = keypack

    # Quick drop of all leading zeros from each of the blocks by converting each string block into an integer
    # and then back into string values
    decipher = str(int(''.join(ciphertext)))

    # Separates the encrypted string into individual list elements, and then combines into blocks of size n
    decipher = [str(''.join(idx).strip()) for idx in ciphertext]
    while '' in decipher:
        decipher.remove('')
    decipher = [(ciphertext[i:i + (len(str(n)))]) for i in range(0, len(ciphertext), (len(str(n))))]
    if decipher[-1].startswith(' '):
        decipher.pop(-1)

    # Decrypts each block by converting ASCii values based on (a^b)^c mod m
    decipher = [str(pow(int(char), key, n)) for char in decipher]

    # Converts each list element from a string to an ASCii integer and then to its character value
    plaintext = [chr(int(char)) for char in decipher]

    return ''.join(plaintext)

'''
User driven - inputs are checked to make sure they are prime before continuing
'''
prime = False
while prime == False:
    p = int(input('\nChoose a prime \'p\' value (larger than 33): '))
    if isPrime(p) == True and p > 30:
        prime = True
    else:
        print('Invalid: Try one of these: [127, 499, 769, 1051, 2161, 3823, 4903, 5227, 6661, 7877]')


prime = False
while prime == False:
    q = int(input('\nChoose a prime \'q\' value (larger than 33): '))
    if isPrime(q) == True and q > 30:
        prime = True
    else:
        print('Invalid: Try one of these: [173, 373, 733, 1559, 2003, 2801, 4999, 5801, 6229, 7919]')

prime = False
while prime == False:
    e = int(input('\nChoose a prime \'e\' value (larger than 33): '))
    if isPrime(e) == True and e > 30:
        prime = True
    else:
        print('Invalid: Try one of these: [71, 443, 977, 1493, 2269, 3557, 4561, 5059, 6203, 7013]')

plaintext = str(input('\nChoose a letter, word, or phrase: '))

e, d, n = keys(p, q, e)
pubkey = (e, n)
privkey = (d, n)

encrypted = encrypt(pubkey, plaintext)
decrypted = decrypt(privkey, encrypted)

print('\npublic key [Use this key for encryption]: (',e,',', n,')')
print('private key [Use this key for decryption]: (',d,',', n,')')

print('\nEncrypted: ', encrypted)
print('\nDecrypted: ', decrypted)
print()

# # Sample 300-digit primes and a message using letters, numbers, and symbols:

# 203956878356401977405765866929034577280193993314348263094772646453283062722701277632936616063144088173312372882677123879538709400158306567338328279154499698366071906766440037074217117805690872792848149112022286332144876183376326512083574821647933992961249917319836219304274280243803104015000563790123
# 531872289054204184185084734375133399408303613982130856645299464930952178606045848877129147820387996428175564228204785846141207532462936339834139412401975338705794646595487324365194792822189473092273993580587964571659678084484152603881094176995594813302284232006001752128168901293560051833646881436219
# 319705304701141539155720137200974664666792526059405792539680974929469783512821793995613718943171723765238853752439032835985158829038528214925658918372196742089464683960239919950882355844766055365179937610326127675178857306260955550407044463370239890187189750909036833976197804646589380690779463976173

#\/\/ 0 /\/\ P! WOMP! ... ThiS IS a TEST, and %it (works)


Choose a prime 'p' value (larger than 33): 43

Choose a prime 'q' value (larger than 33): 59

Choose a prime 'e' value (larger than 33): 13
Invalid: Try one of these: [71, 443, 977, 1493, 2269, 3557, 4561, 5059, 6203, 7013]

Choose a prime 'e' value (larger than 33): 71

Choose a letter, word, or phrase: Professor Moriarty, the "Napoleon of crime" LIVES!

public key [Use this key for encryption]: ( 71 , 2537 )
private key [Use this key for decryption]: ( 995 , 2537 )

Encrypted:  08670922206119941873027002702061092223111396206109222092078509221897037907752311189703901873231102780293078520542061225818732061031623112061199423110769092220922030187302782311243409080344222611862176

Decrypted:  Professor Moriarty, the "Napoleon of crime" LIVES!

